In [15]:
import matplotlib
%matplotlib inline
import os
import random
# os.environ["KERAS_BACKEND"] = "tensorflow"

import matplotlib.pyplot as plt
import seaborn as sns
import pickle, random, sys, keras
from keras.models import Model
from IPython import display

# sys.path.append("../common")

# from tqdm import tqdm

# K.set_image_dim_ordering('th')

In [9]:
import numpy as np
# import theano as th
# import theano.tensor as T
from keras.utils import np_utils
import keras.models as models
from keras.layers import Input,merge

In [13]:
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import *
from keras.layers.wrappers import TimeDistributed
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, Deconv2D, UpSampling2D
from keras.layers.recurrent import LSTM
from keras.regularizers import *
from keras.layers.normalization import *
from keras.optimizers import *
from keras.datasets import mnist
from keras.utils import np_utils

In [17]:
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

print(np.min(X_train), np.max(X_train))

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


0.0 1.0
X_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples


In [21]:
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val

In [24]:
shp = X_train.shape[1:]
print(shp)

(1, 28, 28)


In [33]:
dropout_rate = 0.25
# Optim

opt = Adam(lr=1e-3)
dopt = Adam(lr=1e-4)
#opt = Adam(lr=1e-3)
#opt = Adamax(lr=1e-4)
#opt = Adam(lr=0.0002)
#opt = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999, epsilon=1e-08)
nch = 200

In [48]:
# Build Generative model ...
nch = 200
g_input = Input(shape=[100])
H = Dense(nch*14*14, kernel_initializer='glorot_normal')(g_input)
H = BatchNormalization()(H)
H = Activation('relu')(H)
H = Reshape( [nch, 14, 14] )(H)
H = UpSampling2D(size=(2, 2))(H)
H = Convolution2D(int(nch/2), 3, 3, border_mode='same', kernel_initializer='glorot_uniform')(H)
H = BatchNormalization()(H)
H = Activation('relu')(H)
H = Convolution2D(int(nch/4), 3, 3, border_mode='same', kernel_initializer='glorot_uniform')(H)
H = BatchNormalization()(H)
H = Activation('relu')(H)
H = Convolution2D(1, 1, 1, border_mode='same', kernel_initializer='glorot_uniform')(H)
g_V = Activation('sigmoid')(H)
generator = Model(g_input,g_V)
generator.compile(loss='binary_crossentropy', optimizer=opt)
generator.summary()

/Users/chad/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(100, (3, 3), kernel_initializer="glorot_uniform", padding="same")`
/Users/chad/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (3, 3), kernel_initializer="glorot_uniform", padding="same")`
/Users/chad/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (1, 1), kernel_initializer="glorot_uniform", padding="same")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 100)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 39200)             3959200   
_________________________________________________________________
batch_normalization_18 (Batc (None, 39200)             156800    
_________________________________________________________________
activation_18 (Activation)   (None, 39200)             0         
_________________________________________________________________
reshape_15 (Reshape)         (None, 200, 14, 14)       0         
_________________________________________________________________
up_sampling2d_15 (UpSampling (None, 400, 28, 14)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 400, 28, 100)      12700     
__________

In [50]:
# Build Discriminative model ...
d_input = Input(shape=shp)
H = Convolution2D(256, 5, 5, subsample=(2, 2), border_mode = 'same', activation='relu')(d_input)
H = LeakyReLU(0.2)(H)
H = Dropout(dropout_rate)(H)
H = Convolution2D(512, 5, 5, subsample=(2, 2), border_mode = 'same', activation='relu')(H)
H = LeakyReLU(0.2)(H)
H = Dropout(dropout_rate)(H)
H = Flatten()(H)
H = Dense(256)(H)
H = LeakyReLU(0.2)(H)
H = Dropout(dropout_rate)(H)
d_V = Dense(2,activation='softmax')(H)
discriminator = Model(d_input,d_V)
discriminator.compile(loss='categorical_crossentropy', optimizer=dopt)
discriminator.summary()

# Freeze weights in the discriminator for stacked training
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val
make_trainable(discriminator, False)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 1, 28, 28)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 1, 14, 256)        179456    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 1, 14, 256)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 1, 14, 256)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 1, 7, 512)         3277312   
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 1, 7, 512)         0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 1, 7, 512)         0         
__________

/Users/chad/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 5), activation="relu", strides=(2, 2), padding="same")`
  app.launch_new_instance()
/Users/chad/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (5, 5), activation="relu", strides=(2, 2), padding="same")`


In [60]:
# Build stacked GAN model
gan_input = Input(shape=[100])
H = generator(gan_input)
gan_V = discriminator(H)
# GAN = Model(gan_input, gan_V)
# GAN.compile(loss='categorical_crossentropy', optimizer=opt)
# GAN.summary()

In [59]:
H

<tf.Tensor 'model_2_6/activation_21/Sigmoid:0' shape=(?, 400, 28, 1) dtype=float32>

In [63]:
discriminator(H)

ValueError: number of input channels does not match corresponding dimension of filter, 1 != 28